## Preparing Data for Machine Learning


### Data Import and Loading

In [45]:
import pandas as pd
import numpy as np
import copy, math
import os

In [46]:
username = os.getlogin()
file_name = 'Amazon Sale Report.csv'

possible_paths = [
    os.path.join(os.getcwd(), file_name),  # Current working directory
    f"C:\\Users\\{username}\\Downloads\\{file_name}"  # Downloads folder
]

data = None

for path in possible_paths:
    try:
        data = pd.read_csv(path)
        print(f"✅ Successfully loaded file from: {path}")
        break
    except FileNotFoundError:
        continue

if data is None:
    print("❌ File not found in either location.")

print(f"Loaded {data.shape[0]} records")
print(data.head())


✅ Successfully loaded file from: c:\Users\thech\Documents\My Learning\AISOC\Amazon Sale Report.csv
Loaded 128975 records
   index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  \
0   Merchant      Amazon.in           Standard   SET389   SET389-KR-NP-S   
1   Merchant      Amazon.in           Standard  JNE3781  JNE3781-KR-XXXL   
2     Amazon      Amazon.in          Expedited  JNE3371    JNE3371-KR-XL   
3   Merchant      Amazon.in           Standard    J0341       J0341-DR-L   
4     Amazon      Amazon.in          Expedited  JNE3671  JNE

C:\Users\thech\AppData\Local\Temp\ipykernel_2348\1739949686.py:13: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [47]:
# Exploring the data
print(f"Shape: {data.shape}")
print(f"Columns: {list(data.columns)}")

# Check for problems
print(f"\n🔍 Data Quality Check:")
print(f"Missing values: {data.isnull().sum().sum()}")
print(f"Duplicates: {data.duplicated().sum()}")

Shape: (128975, 24)
Columns: ['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ', 'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids', 'B2B', 'fulfilled-by', 'Unnamed: 22']

🔍 Data Quality Check:
Missing values: 210495
Duplicates: 0


In [48]:
# Converting column to a proper datetime type
date_cols = []  

for col in data.columns:
    if 'date' in col.lower() or 'time' in col.lower():
        date_cols.append(col)  # collect all matching columns

if date_cols:  # if list is not empty
    for col in date_cols:
        try:
            data[col] = pd.to_datetime(data[col])
            print(f"✅ Converted '{col}' to datetime")
        except Exception as e:
            print(f"⚠️ Failed to convert '{col}' to datetime: {e}")
else:
    print("No date/time columns found")


✅ Converted 'Date' to datetime


C:\Users\thech\AppData\Local\Temp\ipykernel_2348\3809867187.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[col] = pd.to_datetime(data[col])


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   Order ID            128975 non-null  object        
 2   Date                128975 non-null  datetime64[ns]
 3   Status              128975 non-null  object        
 4   Fulfilment          128975 non-null  object        
 5   Sales Channel       128975 non-null  object        
 6   ship-service-level  128975 non-null  object        
 7   Style               128975 non-null  object        
 8   SKU                 128975 non-null  object        
 9   Category            128975 non-null  object        
 10  Size                128975 non-null  object        
 11  ASIN                128975 non-null  object        
 12  Courier Status      122103 non-null  object        
 13  Qty                 128975 no

### - Data Profiling and Summary Statistics

In [50]:
data.head(3)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN


In [51]:
data['fulfilled-by'].unique()

array(['Easy Ship', nan], dtype=object)

In [52]:
data['Courier Status'].unique()

array([nan, 'Shipped', 'Cancelled', 'Unshipped'], dtype=object)

In [53]:
data['Courier Status'].fillna("Not Completed", inplace=True)

C:\Users\thech\AppData\Local\Temp\ipykernel_2348\2808219036.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Courier Status'].fillna("Not Completed", inplace=True)


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   Order ID            128975 non-null  object        
 2   Date                128975 non-null  datetime64[ns]
 3   Status              128975 non-null  object        
 4   Fulfilment          128975 non-null  object        
 5   Sales Channel       128975 non-null  object        
 6   ship-service-level  128975 non-null  object        
 7   Style               128975 non-null  object        
 8   SKU                 128975 non-null  object        
 9   Category            128975 non-null  object        
 10  Size                128975 non-null  object        
 11  ASIN                128975 non-null  object        
 12  Courier Status      128975 non-null  object        
 13  Qty                 128975 no

In [55]:
data[data.duplicated()]

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22


In [56]:
data['Order ID'].value_counts()

Order ID
171-5057375-2831560    12
403-4984515-8861958    12
404-9932919-6662730    11
403-0173977-3041148    11
408-3317403-1729937    10
                       ..
403-8464685-9792368     1
407-2378184-3732358     1
403-0816822-3188305     1
402-8261465-0622733     1
408-7436540-8728312     1
Name: count, Length: 120378, dtype: int64

In [57]:
data.loc[data['Order ID'] == '403-4984515-8861958', ['Order ID', 'SKU', 'Date', 'Fulfilment', 'Category', 'Status', 'Amount']]


,Order ID,SKU,Date,Fulfilment,Category,Status,Amount
31737,403-4984515-8861958,JNE3792-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,432.0
31738,403-4984515-8861958,JNE3764-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,487.0
31739,403-4984515-8861958,JNE3503-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,318.0
31740,403-4984515-8861958,J0308-DR-XXXL,2022-04-11,Amazon,Western Dress,Shipped,665.0
31741,403-4984515-8861958,JNE3702-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,342.0
31742,403-4984515-8861958,JNE3741-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,432.0
31743,403-4984515-8861958,JNE3634-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,511.0
31744,403-4984515-8861958,SET433-KR-NP-XXXL,2022-04-11,Amazon,Set,Shipped,666.0
31745,403-4984515-8861958,JNE3636-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,459.0
31746,403-4984515-8861958,JNE3633-KR-XXXL,2022-04-11,Amazon,kurta,Shipped,459.0


In [58]:
unique_data = data['Unnamed: 22'].unique()

In [59]:
print(unique_data)

[nan False]


In [60]:
data['currency'].fillna("INR", inplace=True)

C:\Users\thech\AppData\Local\Temp\ipykernel_2348\1059774145.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['currency'].fillna("INR", inplace=True)


In [61]:
location_check = data[['ship-state', 'ship-city', 'ship-postal-code', 'ship-country']]

In [62]:
location_check[location_check['ship-state'].isnull()]

,ship-state,ship-city,ship-postal-code,ship-country
1871,NaN,NaN,NaN,NaN
1872,NaN,NaN,NaN,NaN
8752,NaN,NaN,NaN,NaN
11215,NaN,NaN,NaN,NaN
13252,NaN,NaN,NaN,NaN
15688,NaN,NaN,NaN,NaN
16787,NaN,NaN,NaN,NaN
18351,NaN,NaN,NaN,NaN
22930,NaN,NaN,NaN,NaN
24986,NaN,NaN,NaN,NaN


In [63]:
data['fulfilled-by'].value_counts()

fulfilled-by
Easy Ship    39277
Name: count, dtype: int64

In [64]:
data.drop(['fulfilled-by'], axis=1, inplace=True)

In [65]:
data['ship-country'].unique()

array(['IN', nan], dtype=object)

In [66]:
data['Unnamed: 22'].unique()

array([nan, False], dtype=object)

In [67]:
data.fillna({'Unnamed: 22': 'unknown'}, inplace=True)

In [68]:
data.drop(['promotion-ids'], axis=1, inplace=True)

In [69]:
data.dropna(subset=['ship-state', 'ship-city', 'ship-postal-code', 'ship-country'], inplace=True)

In [70]:
data.drop(['ship-country'], axis=1, inplace=True)

In [71]:
location_check['ship-city'].value_counts()

ship-city
BENGALURU              11217
HYDERABAD               8074
MUMBAI                  6126
NEW DELHI               5795
CHENNAI                 5421
                       ...  
shimoga                    1
Bangluru                   1
Dombiwali                  1
Begumpet, Hyderabad        1
Badlapur , Thane           1
Name: count, Length: 8955, dtype: int64

In [72]:
location_check[location_check['ship-city'] == 'BENGALURU']

,ship-state,ship-city,ship-postal-code,ship-country
1,KARNATAKA,BENGALURU,560085.0,IN
19,KARNATAKA,BENGALURU,560017.0,IN
21,KARNATAKA,BENGALURU,560040.0,IN
28,KARNATAKA,BENGALURU,560037.0,IN
53,KARNATAKA,BENGALURU,560047.0,IN
...,...,...,...,...
128852,KARNATAKA,BENGALURU,560024.0,IN
128912,KARNATAKA,BENGALURU,560037.0,IN
128939,KARNATAKA,BENGALURU,560035.0,IN
128941,KARNATAKA,BENGALURU,560102.0,IN


In [73]:
location_check

,ship-state,ship-city,ship-postal-code,ship-country
0,MAHARASHTRA,MUMBAI,400081.0,IN
1,KARNATAKA,BENGALURU,560085.0,IN
2,MAHARASHTRA,NAVI MUMBAI,410210.0,IN
3,PUDUCHERRY,PUDUCHERRY,605008.0,IN
4,TAMIL NADU,CHENNAI,600073.0,IN
...,...,...,...,...
128970,TELANGANA,HYDERABAD,500013.0,IN
128971,HARYANA,GURUGRAM,122004.0,IN
128972,TELANGANA,HYDERABAD,500049.0,IN
128973,Gujarat,Halol,389350.0,IN


In [74]:
data[data['Amount'].isna()][['Amount', 'Qty', 'SKU', 'Status', 'ship-city']].head(120)

,Amount,Qty,SKU,Status,ship-city
8,NaN,0,SET200-KR-NP-A-XXXL,Cancelled,HYDERABAD
29,NaN,0,JNE2132-KR-398-XXXL,Cancelled,GUWAHATI
65,NaN,0,JNE3373-KR-XXL,Cancelled,Dahod
84,NaN,0,JNE3510-KR-M,Cancelled,HYDERABAD
95,NaN,0,JNE3405-KR-L,Cancelled,PUNE
...,...,...,...,...,...
1615,NaN,0,JNE3518-KR-M,Shipped,RISHIKESH
1617,NaN,0,SET397-KR-NP-XXXL,Cancelled,BHILAI
1642,NaN,0,JNE3581-KR-M,Cancelled,RISHIKESH
1643,NaN,0,JNE3518-KR-M,Cancelled,RISHIKESH


In [75]:
mean_amount_per_sku = data.groupby('SKU')['Amount'].transform('mean')

In [76]:
mean_amount_per_sku

0          633.765714
1          416.050495
2          332.553077
3          806.907556
4          626.720000
             ...     
128970     468.396806
128971    1032.800000
128972     736.793478
128973    1124.000000
128974     652.285000
Name: Amount, Length: 128942, dtype: float64

In [77]:
data['Amount'] = data['Amount'].fillna(mean_amount_per_sku)

In [78]:
sku_missing = data.groupby('SKU')['Amount'].apply(lambda x: x.isna().all())
print(sku_missing[sku_missing])


SKU
BL020-71RED             True
BL091-XL                True
BL100-XXL               True
BTM002-B-XXL            True
BTM030-NP-S             True
J0024-KR-XXXL           True
J0034-SET-S             True
J0073-KR-S              True
J0086-DR-XL             True
J0100-DR-S              True
J0155-KR-XS             True
J0184-KR-A-L            True
J0184-KR-S              True
J0250-SKD-XS            True
J0374-KR-M              True
JNE2058-KR-339-L        True
JNE2145-KR-A-L          True
JNE2251-KR-537-S        True
JNE2294-KR-A-XS         True
JNE3225-KR-XXXL         True
JNE3311-KR-A-XS         True
JNE3364-KR-1051-XS      True
JNE3366-KR-1053-A-XS    True
JNE3403-KR-XXL          True
JNE3441-KR-E-M          True
JNE3447-KR-XL           True
JNE3483-KR-M            True
JNE3557-KR-XL           True
JNE3662-TP-M            True
JNE3780-KR-XS           True
JNE3891-TP-XXL          True
JNE3897-KR-M            True
JNE3909-KR-L            True
SAR060                  True
SAR182    

In [79]:
data.loc[data['Amount'].isna(), ['Amount', 'SKU']]

,Amount,SKU
363,NaN,SET226-KR-PP-L
3632,NaN,JNE2058-KR-339-L
3634,NaN,JNE3311-KR-A-XS
3637,NaN,BL020-71RED
3971,NaN,J0250-SKD-XS
6744,NaN,BTM002-B-XXL
6793,NaN,BTM002-B-XXL
6802,NaN,BTM002-B-XXL
6832,NaN,BTM002-B-XXL
7057,NaN,J0155-KR-XS


In [80]:
data = data.dropna(subset=['Amount'])

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128898 entries, 0 to 128974
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128898 non-null  int64         
 1   Order ID            128898 non-null  object        
 2   Date                128898 non-null  datetime64[ns]
 3   Status              128898 non-null  object        
 4   Fulfilment          128898 non-null  object        
 5   Sales Channel       128898 non-null  object        
 6   ship-service-level  128898 non-null  object        
 7   Style               128898 non-null  object        
 8   SKU                 128898 non-null  object        
 9   Category            128898 non-null  object        
 10  Size                128898 non-null  object        
 11  ASIN                128898 non-null  object        
 12  Courier Status      128898 non-null  object        
 13  Qty                 128898 non-nul

## Implement Linear Regression Without Using Libraries

##### FEATURE ENGINEERING

In [82]:
# Find target column (sales/amount/revenue)
target_col = None
possible_targets = ['sales', 'amount', 'revenue', 'total'] 
for col in data.columns:
    if col.lower() in possible_targets:
        target_col = col
        break

if target_col is None:
    #Use first numeric column as dataset
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    target_col = numeric_cols[0]

print(f"✅ Target variable: '{target_col}'")

✅ Target variable: 'Amount'


In [83]:
# Create business-smart features from date column
if date_cols:
    for col in date_cols:
        data[f'{col}_Month'] = data[col].dt.month
        data[f'{col}_Is_Holiday_Season'] = data[f'{col}_Month'].isin([11,12]).astype(int)
        data[f'{col}_Is_Weekend'] = data[col].dt.dayofweek.isin([5,6]).astype(int)
    print("✅ Created time-based features for all date columns")

# Encode categories
categorical_cols = data.select_dtypes(include=['object']).columns
categorical_cols = [col for col in categorical_cols if col not in date_cols]

for col in categorical_cols:
    n_unique = data[col].nunique()
    if n_unique <= 20: 
        # one-hot encode columns with less than 20 unique values
        data = pd.get_dummies(data, columns=[col], drop_first=True)
        print(f"✅ One-hot encoded '{col}' with {n_unique} categories")
    else:
        # Frequency encode columns with more than 20 unique values
        freq_encoding = data[col].value_counts() / len(data)
        data[col + '_freq_enc'] = data[col].map(freq_encoding)
        data.drop(columns=[col], inplace=True)
        print(f"✅ Frequency encoded '{col}' with {n_unique} categories")

# Converting bool columns to 0/1 for model training without using external ML libraries
bool_cols = data.select_dtypes(include=['bool']).columns
data[bool_cols] = data[bool_cols].astype(int)


print(f"Engineered: {len(data.columns)} features")

✅ Created time-based features for all date columns
✅ Frequency encoded 'Order ID' with 120311 categories
✅ One-hot encoded 'Status' with 13 categories
✅ One-hot encoded 'Fulfilment' with 2 categories
✅ One-hot encoded 'Sales Channel ' with 2 categories
✅ One-hot encoded 'ship-service-level' with 2 categories
✅ Frequency encoded 'Style' with 1373 categories
✅ Frequency encoded 'SKU' with 7157 categories
✅ One-hot encoded 'Category' with 9 categories
✅ One-hot encoded 'Size' with 11 categories
✅ Frequency encoded 'ASIN' with 7153 categories
✅ One-hot encoded 'Courier Status' with 4 categories
✅ One-hot encoded 'currency' with 1 categories
✅ Frequency encoded 'ship-city' with 8954 categories
✅ Frequency encoded 'ship-state' with 69 categories
✅ One-hot encoded 'Unnamed: 22' with 2 categories
Engineered: 52 features


### MODEL TRAINING

In [84]:
# Prepare the data for the model by dropping target and date columns

columns_to_drop = [target_col]
if date_cols:
    columns_to_drop.extend([col for col in date_cols if col in data.columns])

X = data.drop(columns_to_drop, axis=1)
y = data[target_col]

# Splitting the data into test and train 
indices = np.arange(len(X))
np.random.seed(42)
np.random.shuffle(indices)

# Calculate split point
split_point = int(0.8 * len(X))

# Split into train/test
train_idx, test_idx = indices[:split_point], indices[split_point:]
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# print(f"Features for modeling: {list(X.columns)}")
print(f"Training on {len(X)} samples with {len(X.columns)} features")
m = X_train.shape[0]
print(f"Number of training examples is: {m}")


Training on 128898 samples with 50 features
Number of training examples is: 103118


In [85]:
# Compute the cost function 

def compute_cost(X, y, w, b):
    m = X.shape[0]
    cost = 0.0

    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #scalar 
        cost = cost + (f_wb_i - y[i])**2       #scalar
    cost = cost / (2 * m)                      #scalar    
    return cost


In [86]:
# Compute the Gradient

def compute_gradient(X, y, w, b):
    m,n = X.shape
    dj_dw = np.zeros((n,))  #vector
    dj_db = 0.              #scalar  

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i] 
        for j in range(n):
            dj_dw[j] += err * X[i, j]
        dj_db = dj_db + err 

    dj_dw = dj_dw / m
    dj_db = dj_db / m 

    return dj_db, dj_dw


In [87]:
# Compute Gradient Descent
def gradient_descent(X, y, w_in, b_in, compute_cost, compute_gradient, alpha, num_iters): 

    J_history = []
    w = w_in
    b = b_in

    for i in range(num_iters):
        # Calculate the gradient and update the parameters
        dj_db,dj_dw = compute_gradient(X, y, w, b) 

        # Update Parameters using w, b, alpha and gradient
        w -= alpha * dj_dw
        b -= alpha * dj_db 

        # Save cost J at each iteration
        if i<200:      
            J_history.append(compute_cost(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history 

In [88]:
b_init = 0
w_init = np.zeros(X_train.shape[1])

iterations = 500
alpha = 5.0e-7

X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

w_final, b_final, J_hist = gradient_descent(
    X_train_np, y_train_np, w_init, b_init,
    compute_cost, compute_gradient, alpha, iterations
)

print(f"b found: {b_final:.2f}")
print(f"w found: {w_final}")
print(f"Final cost: {J_hist[-1]:.2f}")

Iteration    0: Cost 2900075802491877.00   


C:\Users\thech\AppData\Local\Temp\ipykernel_2348\2575823737.py:9: RuntimeWarning: overflow encountered in scalar power
  cost = cost + (f_wb_i - y[i])**2       #scalar


Iteration   50: Cost      inf   


C:\Users\thech\AppData\Local\Temp\ipykernel_2348\4093350604.py:11: RuntimeWarning: overflow encountered in scalar add
  dj_dw[j] += err * X[i, j]
C:\Users\thech\AppData\Local\Temp\ipykernel_2348\4093350604.py:11: RuntimeWarning: invalid value encountered in scalar multiply
  dj_dw[j] += err * X[i, j]


Iteration  100: Cost      nan   
Iteration  150: Cost      nan   
Iteration  200: Cost      nan   
Iteration  250: Cost      nan   
Iteration  300: Cost      nan   
Iteration  350: Cost      nan   
Iteration  400: Cost      nan   
Iteration  450: Cost      nan   
b found: nan
w found: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
Final cost: nan
